In [8]:
import pandas as pd
import json

In [2]:
with open("slam-data/es_en.slam.20190204.dev", "r") as f:
    raw_data = f.read()

In [3]:
with open("slam-data/es_en.slam.20190204.dev.key", "r") as f:
    raw_success_data = f.read()

In [4]:
success_key = {}
for x in raw_success_data.split("\n"):
    entry = x.split()
    if len(entry) == 2:
        success_key[entry[0]] = eval(entry[1])

In [5]:
prompts = raw_data.split("# prompt:")[1:] # Removed Promptless session

In [6]:
data = []
for prompt in prompts:
    sessions = prompt[prompt.index("\n"):].split("# user:")

    for session in sessions: 
        if len(session) <= 1:
            continue
        
        
        exercises = session[session.index("\n"):].split("\n") 
        exercises = list(filter(lambda x: len(x) > 0, exercises))
    
        word_tokens = []
        word_flags = []
        word_parts = []
        
        for exercise in exercises:
            exercise_data = exercise.split() 
            
            word_tokens.append(exercise_data[1])
            word_flags.append(success_key[exercise_data[0]])
            word_parts.append(exercise_data[2])
    
        sessionId = exercise[:10]
        userId = session[:session.index("\n")].split()[0] 
        days = session[:session.index("\n")].split()[2] 

        data.append({
            "sessionId": sessionId,
            "userId": userId,
            "words": word_tokens,
            "flags": word_flags,
            "parts": word_parts,
            "day": eval(days[len("days:"):])
        })

In [8]:
df = pd.DataFrame(data)

In [10]:
df.to_csv("data.csv")

In [9]:
with open("session-data.json", "w") as f:
    json.dump(data, f)